# Importing the Dependencies

In [1]:
import pandas as pd  
import numpy as np  
import os
import json
import traceback
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

/Users/dhruv/Desktop/Machine_Learning/Projects/MCQ_Generator/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv() # Loading the env to fetch environment variables

KEY = os.getenv("OPENAI_API_KEY")

In [3]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.3)

/Users/dhruv/Desktop/Machine_Learning/Projects/MCQ_Generator/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Importing the required libraries and modules from Langchain

In [4]:
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 
from langchain.chains import LLMChain 
from langchain.chains import SequentialChain 
from langchain.callbacks import get_openai_callback 
import PyPDF2

# Quiz Generation - Prompt and Chain Initialization

In [5]:
RESPONSE_JSON = {
    "1": {
        "tfq": "true false question",
        "options": {
            "a": "True",
            "b": "False"
        },
        "correct": "correct answer",
    },
    "2": {
        "tfq": "true false question",
        "options": {
            "a": "True",
            "b": "False"
        },
        "correct": "correct answer",
    },
    "3": {
       "tfq": "true false question",
        "options": {
            "a": "True",
            "b": "False"
        },
        "correct": "correct answer",
    },
}


In [6]:
TEMPLATE = """
Text: {text}
You are an expert True False Question maker. Given the above text, it is your job to \
create a quiz of {number} True/False questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response likeRESPONSE_JSON below and use it as a guide. \
Ensure to make {number} TFQs

### RESPONSE_JSON
{response_json}
"""

# Here, tone implies difficulty level



quizGen_prompt_template = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)


quiz_chain = LLMChain(llm=llm,prompt=quizGen_prompt_template,output_key="quiz",verbose=True)

# Answer Evalusation Template Initiliazation

In [7]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a True False Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_TFQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

quizEval_prompt_template = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

review_chain = LLMChain(llm=llm,prompt=quizEval_prompt_template,output_key="review",verbose=True)

# Sequential Chain Generation
Let us now generate a chain of prompts and answers for the quiz generation task. Here, both the above chains will be combined to generate a single chain and give us the final output as desired.

In [8]:
evaluation_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                        output_variables=["quiz","review"],verbose=True)

# Data Loading

In [9]:
file_path = '../data.txt'
with open(file_path,'r') as file:
    TEXT = file.read()

print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]
Machine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.
The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised l

In [10]:
# Now, let us serialize the above python dictionary into a json-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"tfq": "true false question", "options": {"a": "True", "b": "False"}, "correct": "correct answer"}, "2": {"tfq": "true false question", "options": {"a": "True", "b": "False"}, "correct": "correct answer"}, "3": {"tfq": "true false question", "options": {"a": "True", "b": "False"}, "correct": "correct answer"}}'

# Callback Function for Quiz Generation
Now, we will define a callback function that will be used to generate the quiz questions and answers. This function will be called by the chain to generate the quiz questions and answers. By this, we can track the tokens used in the prompt and the answer and get the cost of the prompt and the answer. Read more here [here](https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking).

In [11]:
NUMBER = 5
SUBJECT = "Machine Learning"
TONE = "Simple"

In [12]:
with get_openai_callback() as cb:
    response=evaluation_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/Users/dhruv/Desktop/Machine_Learning/Projects/MCQ_Generator/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]
Machine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.
The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mini

Let us get the token usage of the prompt and the answer.

In [13]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost in USD:{cb.total_cost}")

Total Tokens:1144
Prompt Tokens:803
Completion Tokens:341
Total Cost in USD:0.0018865000000000002


Let us now try to get the response from the chain and see our quiz.

In [14]:
response

{'text': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\nMachine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through un

In [15]:
quiz = response.get('quiz')
quiz = json.loads(quiz)
quiz

{'1': {'tfq': 'Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms.',
  'options': {'a': 'True', 'b': 'False'},
  'correct': 'True'},
 '2': {'tfq': 'Artificial neural networks have not been able to surpass many previous approaches in performance.',
  'options': {'a': 'True', 'b': 'False'},
  'correct': 'False'},
 '3': {'tfq': 'Data mining is a field of study focusing on exploratory data analysis through supervised learning.',
  'options': {'a': 'True', 'b': 'False'},
  'correct': 'False'},
 '4': {'tfq': 'PAC learning provides a framework for describing machine learning from a theoretical viewpoint.',
  'options': {'a': 'True', 'b': 'False'},
  'correct': 'True'},
 '5': {'tfq': 'Computational statistics is not an important source of machine learning methods.',
  'options': {'a': 'True', 'b': 'False'},
  'correct': 'False'}}

Let us now convert the response to a DataFrame and display the quiz.

In [18]:
quiz_table_data = []
for key, value in quiz.items():
    tfq = value["tfq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"TFQ": tfq, "Choices": options, "Correct": correct})

In [20]:
quiz = pd.DataFrame(quiz_table_data)
quiz

,TFQ,Choices,Correct
0,Machine learning is a field of study in artifi...,a: True | b: False,True
1,Artificial neural networks have not been able ...,a: True | b: False,False
2,Data mining is a field of study focusing on ex...,a: True | b: False,False
3,PAC learning provides a framework for describi...,a: True | b: False,True
4,Computational statistics is not an important s...,a: True | b: False,False


Now finally, we can export the above DataFrame to a csv file.

In [22]:
quiz.to_csv("ML_TF_Quiz.csv",index= False)